In [152]:
import cadquery as cq
from cadquery import Workplane, Assembly, Location, Vector, Color, exporters
# from cq_server.ui import ui, show_object
# from ocp_vscode import *

from ocp_vscode import show, show_object, reset_show, set_port, set_defaults, get_defaults
set_port(3939)

In [153]:
um = 0.001
reset_show()
wp = Workplane("XY")
bar_width = 300*um
bar_length = 2000*um
bar_thickness = 20*um
gap = 2000*um

top_bar_width = 2300*um
top_bar_length = 300*um

c_bridge_width = 300*um
c_bridge_length = 450*um

c_pad_width = 700*um # This is a parameter

signal_pad_length = 550*um

via_r = (c_bridge_width/2)
via_thickness = 200*um # This is a parameter

left_bar = (
    wp.rect(bar_width, bar_length,centered=False)
    .extrude(bar_thickness)
)
right_bar = (
    wp.move(gap, 0)
    .rect(bar_width, bar_length,centered=False)
    .extrude(bar_thickness)
)
top_bar = (
    wp.moveTo(0,0)
    .move(0,bar_length)
    .rect(top_bar_width,top_bar_length,centered=False)
    .extrude(bar_thickness)
)
c_bridge = (
    wp.move(gap/2,bar_length)
    .rect(c_bridge_width,-c_bridge_length,centered=False)
    .extrude(bar_thickness)
)
c_pad = (
    wp.moveTo((top_bar_width/2),(bar_length - c_bridge_length)-c_pad_width/2)
    .rect(c_pad_width,c_pad_width,centered=True)
    .extrude(bar_thickness)

)

result = (
    left_bar
    .union(right_bar)
    .union(top_bar)
    .union(c_bridge)
    .union(c_pad)
)

show_object(result, name="test")

c


# The Bottom CI Signal Pad and Connection

In [154]:

signal_pad = (
    wp.move(gap/2,0)
    .rect(bar_width,signal_pad_length,centered=False)
    .extrude(bar_thickness)
)

show_object(signal_pad, name="signal_pad")



cc


# Make the via 

In [155]:
via = (
    wp.moveTo((top_bar_width/2),signal_pad_length)
    .circle(via_r)
    .extrude(via_thickness+bar_thickness)
)
show_object(via, name="via")

ccc


# The top electrode and connector

In [156]:
top_electrode = (
    wp.workplane(offset=200*um)
    .moveTo((top_bar_width/2),(bar_length - c_bridge_length)-c_pad_width/2)
    .rect(c_pad_width, c_pad_width, centered=True)
    .extrude(bar_thickness)
)
show_object(top_electrode, name="top_electrode")

top_connector = (
    wp.workplane(offset=200*um)
    .moveTo((top_bar_width/2 -c_bridge_width/2 ),(bar_length - c_bridge_length - c_pad_width))
    .rect(c_bridge_width,-(bar_length-c_bridge_length-c_pad_width-signal_pad_length),centered=False)
    .extrude(bar_thickness)
)
show_object(top_connector, name="top_connector")

cccc


ccccc


# The DI Layer

In [157]:
# DI Starts at XY plane
# DI's reference point is at the top left corner of the c_pad
c_pad_tl = (
    c_pad.faces(">Z")
    .vertices("<X and >Y")
    .val()
    .Center()
)
DI = (
    wp.moveTo(c_pad_tl.x - 100*um, c_pad_tl.y + 100*um)
    .rect(c_pad_width+200*um,-(bar_length - c_bridge_length - bar_width),centered=False)
    .extrude(via_thickness)
)

show_object(DI, name="DI")

ccccc+

In [158]:
tmp_assamble = Assembly()
tmp_assamble.add(result, name="bot_CI")
tmp_assamble.add(signal_pad, name="signal_CI")
tmp_assamble.add(via, name="via")
tmp_assamble.add(top_electrode, name="top_electrode")
tmp_assamble.add(top_connector, name="top_connector")
tmp_assamble.add(DI, name="DI")

tmp_assamble.save("tmp_assamble.step")